In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm

In [2]:
list_api = ['https://api.worldbank.org/v2/en/indicator/NY.GDP.MKTP.CD?downloadformat=excel', #GDP
            'https://api.worldbank.org/v2/en/indicator/NY.GDP.MKTP.KD.ZG?downloadformat=excel', #GDP Growth
            'https://api.worldbank.org/v2/en/indicator/NY.GDP.PCAP.CD?downloadformat=excel', #GDP per capital
            'https://api.worldbank.org/v2/en/indicator/NY.GDP.PCAP.KD.ZG?downloadformat=excel', #GDP per capital grpwth
            'https://api.worldbank.org/v2/en/indicator/GC.DOD.TOTL.GD.ZS?downloadformat=excel', # debt to gdp
            'https://api.worldbank.org/v2/en/indicator/SP.POP.TOTL?downloadformat=excel', # population
            'https://api.worldbank.org/v2/en/indicator/SP.POP.TOTL.FE.ZS?downloadformat=excel', # female
            'https://api.worldbank.org/v2/en/indicator/FP.CPI.TOTL.ZG?downloadformat=excel', #inflation
            'https://api.worldbank.org/v2/en/indicator/FR.INR.RINR?downloadformat=excel', # real interest rate
            'https://api.worldbank.org/v2/en/indicator/FR.INR.DPST?downloadformat=excel', #deposit , lending1
            'https://api.worldbank.org/v2/en/indicator/FR.INR.LEND?downloadformat=excel']

In [3]:
list_df_api = []
for i in tqdm(list_api):
    url= i
    df=pd.read_excel(url, skiprows=3)
    melted_data = pd.melt(df, id_vars=['Country Name', 'Country Code', 'Indicator Name', 'Indicator Code'],
                        var_name='Year', value_name='Value', value_vars=[str(year) for year in range(1960, 2023)])
    melted_data = melted_data[melted_data['Year'].apply(lambda x: x.isdigit())]
    melted_data.reset_index(drop=True, inplace=True)
    melted_data.dropna(inplace=True)
    list_df_api.append(melted_data)


100%|██████████| 11/11 [00:17<00:00,  1.59s/it]


In [35]:
list_df_api[0][list_df_api[0]['Country Name'] == 'Australia']

,Country Name,Country Code,Indicator Name,Indicator Code,Year,Value
13,Australia,AUS,GDP (current US$),NY.GDP.MKTP.CD,1960,1.860656e+10
279,Australia,AUS,GDP (current US$),NY.GDP.MKTP.CD,1961,1.968288e+10
545,Australia,AUS,GDP (current US$),NY.GDP.MKTP.CD,1962,1.992256e+10
811,Australia,AUS,GDP (current US$),NY.GDP.MKTP.CD,1963,2.153984e+10
1077,Australia,AUS,GDP (current US$),NY.GDP.MKTP.CD,1964,2.380112e+10
...,...,...,...,...,...,...
15441,Australia,AUS,GDP (current US$),NY.GDP.MKTP.CD,2018,1.429734e+12
15707,Australia,AUS,GDP (current US$),NY.GDP.MKTP.CD,2019,1.394671e+12
15973,Australia,AUS,GDP (current US$),NY.GDP.MKTP.CD,2020,1.330382e+12
16239,Australia,AUS,GDP (current US$),NY.GDP.MKTP.CD,2021,1.559034e+12


In [28]:
df_eco = list_df_api[0].copy()
for i in (range(len(list_df_api))):
    df = list_df_api[i].copy()
    df.drop(columns='Indicator Code',inplace=True)
    name_column = df['Indicator Name'].iloc[1]
    df.rename(columns={'Value':name_column},inplace=True)
    df_eco = pd.merge(df_eco,df[['Country Code','Year',name_column]],how='outer',on=['Country Code','Year'])

df_eco.drop(columns=['Value','Indicator Name','Indicator Code'],inplace=True)

In [29]:
df_eco.dropna(thresh=8, inplace=True) # drop row ที่มี nan เกิน 8
map_country = df_eco[['Country Name','Country Code']].dropna().drop_duplicates(subset=['Country Name','Country Code'], keep='first') # drop duplicate column
df_eco = pd.merge(df_eco,map_country,how='left',on='Country Code',suffixes=['_x',''])
df_eco.drop(columns=['Country Name_x'],inplace=True)
df_eco.fillna(0,inplace=True)

In [30]:
df_eco

,Country Code,Year,GDP (current US$),GDP growth (annual %),GDP per capita (current US$),GDP per capita growth (annual %),"Central government debt, total (% of GDP)","Population, total","Population, female (% of total population)","Inflation, consumer prices (annual %)",Real interest rate (%),Deposit interest rate (%),Lending interest rate (%),Country Name
0,AUS,1960,1.860656e+10,0.0,1810.597443,0.0,0.0,10276477.0,49.479398,3.728814,0.000000,0.000000,5.000000,Australia
1,AUT,1960,6.592694e+09,0.0,935.460427,0.0,0.0,7047539.0,53.420644,1.945749,0.000000,0.000000,0.000000,Austria
2,BEL,1960,1.165872e+10,0.0,1273.691659,0.0,0.0,9153489.0,50.991840,0.299467,0.000000,0.000000,0.000000,Belgium
3,BFA,1960,3.304428e+08,0.0,69.083195,0.0,0.0,4783259.0,49.090651,7.788162,0.000000,0.000000,0.000000,Burkina Faso
4,BOL,1960,3.732500e+08,0.0,100.673902,0.0,0.0,3707515.0,50.673376,11.533153,0.000000,0.000000,0.000000,Bolivia
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12768,SSD,2020,0.000000e+00,0.0,0.000000,0.0,0.0,10606227.0,50.518955,29.675844,14.551307,0.075830,16.089617,South Sudan
12769,SSD,2021,0.000000e+00,0.0,0.000000,0.0,0.0,10748272.0,50.498238,10.516716,-8.153529,0.037382,16.088483,South Sudan
12770,BTN,2022,0.000000e+00,0.0,0.000000,0.0,0.0,782455.0,47.098555,5.639167,7.755708,3.000000,14.000000,Bhutan
12771,SSD,2022,0.000000e+00,0.0,0.000000,0.0,0.0,10913164.0,50.476892,-6.687321,-49.955193,0.060227,16.293496,South Sudan


In [33]:
country = pd.read_csv('country.csv')
country

,Country,Continent
0,Afghanistan,Asia
1,Albania,Europe
2,Algeria,Africa
3,Andorra,Europe
4,Angola,Africa
...,...,...
191,Venezuela,South America
192,Viet Nam,Asia
193,Yemen,Asia
194,Zambia,Africa


In [31]:
# mapping 
df_eco_1 = pd.merge(df_eco,country,how='left',left_on='Country Name',right_on='Country')
df_eco_2 = df_eco_1[~(df_eco_1['Continent'].isna())]
df_eco_2

,Country Code,Year,GDP (current US$),GDP growth (annual %),GDP per capita (current US$),GDP per capita growth (annual %),"Central government debt, total (% of GDP)","Population, total","Population, female (% of total population)","Inflation, consumer prices (annual %)",Real interest rate (%),Deposit interest rate (%),Lending interest rate (%),Country Name,Country,Continent
0,AUS,1960,1.860656e+10,0.0,1810.597443,0.0,0.0,10276477.0,49.479398,3.728814,0.000000,0.000000,5.000000,Australia,Australia,Oceania
1,AUT,1960,6.592694e+09,0.0,935.460427,0.0,0.0,7047539.0,53.420644,1.945749,0.000000,0.000000,0.000000,Austria,Austria,Europe
2,BEL,1960,1.165872e+10,0.0,1273.691659,0.0,0.0,9153489.0,50.991840,0.299467,0.000000,0.000000,0.000000,Belgium,Belgium,Europe
3,BFA,1960,3.304428e+08,0.0,69.083195,0.0,0.0,4783259.0,49.090651,7.788162,0.000000,0.000000,0.000000,Burkina Faso,Burkina Faso,Africa
4,BOL,1960,3.732500e+08,0.0,100.673902,0.0,0.0,3707515.0,50.673376,11.533153,0.000000,0.000000,0.000000,Bolivia,Bolivia,South America
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12768,SSD,2020,0.000000e+00,0.0,0.000000,0.0,0.0,10606227.0,50.518955,29.675844,14.551307,0.075830,16.089617,South Sudan,South Sudan,Africa
12769,SSD,2021,0.000000e+00,0.0,0.000000,0.0,0.0,10748272.0,50.498238,10.516716,-8.153529,0.037382,16.088483,South Sudan,South Sudan,Africa
12770,BTN,2022,0.000000e+00,0.0,0.000000,0.0,0.0,782455.0,47.098555,5.639167,7.755708,3.000000,14.000000,Bhutan,Bhutan,Asia
12771,SSD,2022,0.000000e+00,0.0,0.000000,0.0,0.0,10913164.0,50.476892,-6.687321,-49.955193,0.060227,16.293496,South Sudan,South Sudan,Africa


In [32]:
df_eco_2.to_csv('economic_data.csv')